<a href="https://colab.research.google.com/github/drawVT/RSSHub/blob/master/CSIM_CogSys_MLP_tutorial_140223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [30860] Cognitive Systems: Theories and Models
## Week 6 - Multi Layer Networks Tutorial


Authors: R. Zucca, G. Maffei, and A. F. Amil

## Part I: Implementing a 3-layer neural network from scratch

#### Import the necessary  modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons
import matplotlib
from matplotlib.colors import ListedColormap
import seaborn as sb
import numpy.random as random

import pandas as pd

# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (5.0, 5.0)

## Generating a toy dataset

We import a dataset from the ones available in the *scikit-learn* package (www.scikit-learn.org). 

Scikit-learn provides many predefined datasets that can be used to test your models.


Some examples: 

- `sklearn.datasets.load_boston` load and return the boston house-prices dataset (regression).

- `sklearn.datasets.load_breast_cancer` Load and return the breast cancer wisconsin dataset (classification).

- `sklearn.datasets.make_circles` Make a large circle containing a smaller circle in 2d (clustering and classification)

Let's use the <b>make_moons</b> one which produces Gaussian data with a spherical decision boundary for binary classification.

In [ ]:
# Set a random seed generator to reproduce the same results
random_state = 0

# Generate a dataset of 200 points from a Gaussian distribution with standar deviation equal to 0.2
# Input:
#     n_points: We generate 200 points
#     noise: Standard deviation of Gaussian noise added to the data.
# Output:
#     X: training dataset
#     y: target labels


n_samples = 200
noise     = 0.2

X, y = make_moons(n_samples=n_samples, noise=noise, random_state=random_state)

# Check the dimensionality of the generated dataset
print('X (samples x features): %d x %d' % (X.shape[0], X.shape[1]))
print('y (labels): %d ' % y.shape)

# Define some colormaps
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])


# Plot the data
plt.figure(figsize=(6,6))
plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=cm_bright)
plt.xticks([])
plt.yticks([])
plt.show()

We now have two classes of datapoints separated by a <b>non linear</b> boundary.

In [ ]:
# This function is used to plot the decision boundary

def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    # Step size in the meshgrid
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), 
                         np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])   # ravel function returns a contiguous flattened array
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=cm, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)

#### As we have already seen, a SLP is not able to correctly classify such a dataset

Minksy and Papert showed that a two layer feed-forward network can overcome many restrictions of single layer networks, but did not present a solution to the problem of how to adjust the weights from input to hidden units.

An answer to this question was provided by Rumelhart, Hinton and Williams (1986):
- the central idea is that errors for the units of the hidden layer are determined by back-propagating the errors of the units of the output layer.

In [ ]:
def perceptron(training_set, target_vector, stopAt=1000, w=np.zeros(3)):
    '''
        A very simple implementation of the perceptron algorithm for two dimensional data.
        Input: 
            training_set  : Data points, an Nx2 vector. 
            target_vector : Classification of the previous data points, an Nx1 vector. 
            stopAt  : Maximum number of iterations (optional).
            w             : Initial vector of weights (by default they are initialized with zeros)
            
        Output: 
            w : Weight vector (i.e., parameters of the best linear separator, y = ax+b)
'''
    # Number of training samples
    N = training_set.shape[0]
    
    # Activation function
    # Expanded formula
    f = lambda x: np.sign(w[0]+w[1]*x[0]+w[2]*x[1])

    ########################
    # Learning loop
    ########################
    i = 0  # counter for examples
    c = 1
    for _ in range(stopAt):
        
        i = np.random.randint(N)
        # Calculate prediction
        y_hat = f(training_set[i,:])
        
        # Error correction
        if(target_vector[i] != y_hat): 
             error = target_vector[i] - y_hat
             w[0] = w[0] + error # correct the bias term
             w[1] = w[1] + error*training_set[i,0] # correct the first weight
             w[2] = w[2] + error*training_set[i,1] # correct the second weight
        i+=1
        if np.mod(i,N)==0:
            #print 'Training epoch:', c 
            c+=1
            i = 0
    return w;

In [ ]:
Xn = X/np.std(X, axis=0)
W = perceptron(X, y, stopAt=10000)

In [ ]:
# Rescale the weights to draw the boundary line
b_new = -W[0]/W[2]
a_new = -W[1]/W[2]
x = np.linspace(-2, 3, 50)
f = lambda x: a_new * x + b_new

plt.figure(figsize=(5,5))
plt.scatter(Xn[:,0], Xn[:,1], c=y, s=30, cmap=plt.cm.Paired)
plt.plot(x,f(x),'b--',label = 'Perceptron solution')
plt.ylim(-2,3)
plt.legend()
plt.show()

## Multilayer Perceptron

A multilayer perceptron (MLP) is a deep, artificial neural network. It is composed of more than one perceptron. 

They are composed of an input layer to receive the signal, an output layer that makes a decision or prediction about the input, and in between those two, an arbitrary number of hidden layers that are the true computational engine of the MLP. MLPs with one hidden layer are capable of approximating any continuous function.


We build a 3-layer neural network with one input layer, one hidden layer, and one output layer. The number of nodes in the input layer is determined by the dimensionality of the data, i.e. 2.

The number of nodes in the output layer is determined by the number of classes we have, i.e.,  1 unit for a binary classification problem, 2 units for multiclass problems. 

The input to the network will be x- and y- coordinates and its output will be the predicted class. Depending on the activation function we can have a fixed or continuous value (representing the probability to belong to a class)

<img src="https://drive.google.com/uc?id=1J0HNZvnKs6koL_wbixsUfqClI2nFe51_" width="600">

<img src="https://drive.google.com/uc?id=1w6INnIehjB56CiICmW9qaIEv7V-HxA00" width="600">

<img src="https://drive.google.com/uc?id=1pbAXUkarhKL1MdSM3Xnpukq3zuSzni7q" width="600">

### Hidden layer
The dimensionality of the hidden layer depends on the specific problem you are dealing with.

More nodes can deal with more complex functions, but at the cost of more computations and the possibility to overfit the data.  Overfitting occurs when a model with high capacity fits the noise in the data instead of the (assumed) underlying relationship.

It has been shown that only one layer of hidden units suffices to approximate any function, provided the activation function of the hidden units is non linear.

## Overfitting problem
### The problem at hands

<img src="https://drive.google.com/uc?id=1XOjpMKazYKLtlraUxAsK__DbkUMRTD3l" width="500">

### A complex model
<img src="https://drive.google.com/uc?id=1fpT-VpnkrZYG2miRvKNczOaM_rBSVMsH" width="500">

### The true (much simpler) model
<img src="https://drive.google.com/uc?id=1dlQBgmH6mGybMvnEJZZy_SJfOxygEx7w" width="500">

### How overfitting affects prediction
<img src="https://drive.google.com/uc?id=1dQioFSsnxX-N8OLy9_pAlfN2QAJ5FjK9" width="600">

## The activation function
Choose an activation function for the hidden layer (i.e, linear rectifier, sigmoid, etc).

### Optimization: learning the parameters that minimize the loss function

Learning the parameters for our network means finding parameters ($W_1, b_1, W_2, b_2$) that *minimize the error* on our training data. 

The amount of error is calculated through the *loss function* (loss for prediction with respect to true labels). The loss function can be considered as a quality measure

### Back-propagation

When a learning pattern is clamped  the activation values are propagated to the output units  and the actual network output is compared with the desired output values  we usually end up with an error in each of the output units.

Let's call this error $e_o$ for a particular output unit $o$.  We have to bring $e_o$ to $0$. 

**Step 1**
The simplest way to achieve this is the **greedy method**: we strive to change the connections in such a way that, next time arounds, the error $e_o$ will be zero for this particular pattern. In order to reduce the error we have to adapt its incoming weights according to 

$$ \delta w_{ho} = (d_o - y_o ) y_h$$

**Step 2**
To adapt the weights from input to hidden units, we again want to apply the delta rule. In this case, we don't have a value for $\delta$ in the hidden units. This is solved by the so called **chain rule**:

- the error of a output unit $o$ is distributed to all the hidden units that it is connected to, weighted by this connection or differently said:
- a hidden unit $h$ receives a delta from each output unit $o$ equal to the delta of that output unit weighted with the weight of the connection between those units. The activation function of the hidden unit is then applied to the delta.

<img src="https://drive.google.com/uc?id=1uBlf2KH73GTnXQf7WZJSum-LC1uACA2L" width="600">

#### How BP works
**Phase 1**
- The input **x** is presented and propagated forward through the network to compute the output value **y** for each unit.
- The output is compared with the desired value **d** resulting in an error signal $\delta$ for each output unit.

**Phase 2**
- The error is passed back to each unit and the weight changes are calculated.

When using a sigmoid function:
- The weight of a connection is adjusted by an amount prop ortional to the pro duct of an error signal on the unit k receiving the input and the output of the unit j sending this signal along the connection

$$\Delta_p w_{jk} = \lambda \delta_k y_j$$

- If the unit is an output unit the error is given by:

$$\delta_o = (\delta_o - y_o) F(s_o)$$

where

$$y = F(s) = 1/(1+e^{-s})$$

in this case:

$$F(s) = y(1-y)$$

and the error is:

$$\delta_o = (d_o -y_o ) y_o (1-y_o)$$

The error signal of a hidden unit is determined recurisvely in terms of error signals of the units to which it directly connects and the weights of those connections:

$$\delta_h = y_h (1-y_h) \sum \delta_o w_{ho}$$

**Learning and momentum**
Learning requires that the chnage in weight is porportional to $dE / dw$. 

The proportionality constant (allowing for infinitesimal steps) is the learning rate $\lambda$. To avoid oscillations the change is made dependent on the past weight change by adding a momentum term.


**Learning per pattern**
The order of pattern presentation is important. To avoid the network to focus on the first few patterns the order of presentation is shuffled. 

### Limits of BP
- Long training process
- Network paralysis (too high values) and local minima (the network get trapped in a local minimum that is not the optimal solution)

<img src="https://drive.google.com/uc?id=13SFWpk1fgWL5PSXVfq30q_tuRI_35sRi" width="600">

### How good are multi-layer FF networks?
The approximation of a network is not perfect. It depends on:
- The learning algorithm and number of iterations.  This determines how good the error on the training set is minimized
- The number of learning samples.  This determines how good the training samples represent the actual function
- The number of hidden units.  This determines the  expressive power  of the network. For  smooth  functions only a few number of hidden units are needed  for wildly  fluctuating functions more hidden units will be needed.

### An example: How our network makes predictions

If $x$ is the 2D input to our network then we calculate our output $\hat{y}$ (also two-dimensional) as follows:

$$
\begin{aligned}
z_1 & = W_1 x + b_1 \\
a_1 & = \tanh(z_1) \\
z_2 & = W_2 a_1 + b_2 \\
a_2 & = \hat{y} = i.e. \mathrm{softmax}(z_2)
\end{aligned}
$$

- $z_i$ is the input of layer $i$
- $a_i$ is the output of layer $i$ after applying the activation function (e.g., tanh for layer 1, softmax for layer 2)
- $W_1, b_1, W_2, b_2$ are  the weights (parameters) of the network, which we need to learn from our training data. 


##### Softmax:
The softmax activation function gives out normalized class probabilities: it takes a vector of real-valued scores (in zz) and squashes it to a vector of values between zero and one that sum to one

### Optimization

The amount of error is calculated through the *loss function* (loss for prediction with respect to true labels). The loss function can be considered as a quality measure and is given by:

$$
\begin{aligned}
L = - \frac{1}{N} \sum_{n \in N} \sum_{i \in C} y_{n,i} \log\hat{y}_{n,i}
\end{aligned}
$$


What this function does is to sum over our training examples and add to the loss if the prediction was incorrect. 

So, **the further away $y$ (the correct labels) and $\hat{y}$ (our predictions) are, the greater our loss will be.** 

To find the parameters that minimize our loss function we use a **gradient descent$^{1}$** to find its minimum, that is we repeatedly evaluate the gradient and then perform a parameter update.

The gradient tells us the direction in which the function has the steepest rate of increase, but it does not tell us how far along this direction we should step.

To keep things simple we are going to use a batch gradient descent with a fixed learning rate. 

As an input, gradient descent needs the gradients (a vector of slopes or the derivatives) of the loss function with respect to the parameters: $\frac{\partial{L}}{\partial{W_1}}$, $\frac{\partial{L}}{\partial{b_1}}$, $\frac{\partial{L}}{\partial{W_2}}$, $\frac{\partial{L}}{\partial{b_2}}$. 

[*The derivative of a function $y = f(x)$ of a variable $x$ is a measure of the rate at which the value $y$ of the function changes with respect to the change of the variable x. It describes the instantaneous rate of change of the variable.*]


To calculate these gradients we use the *backpropagation algorithm*, an efficient way to calculate the gradients starting from the output. 

Applying the backpropagation formula we find the following:

$$
\begin{aligned}
& \delta_3 = y - \hat{y} \\
& \delta_2 = (1 - \tanh^2z_1) \circ \delta_3W_2^T \\
& \frac{\partial{L}}{\partial{W_2}} = a_1^T \delta_3  \\
& \frac{\partial{L}}{\partial{b_2}} = \delta_3\\
& \frac{\partial{L}}{\partial{W_1}} = x^T \delta_2\\
& \frac{\partial{L}}{\partial{b_1}} = \delta_2 \\
\end{aligned}
$$

### Implementation

In [ ]:
# Initialization parameters
num_examples = len(X) # Size of the training set
nn_input_dim = 2      # Dimensionality of the input layer
nn_output_dim = 2     # Dimensionality of the output layer

# Gradient descent parameters 
epsilon = 0.01 # Learning rate for the gradient descent
reg_lambda = 0.01 # Regularization strength

Implement a loss function evaluating the performance of the model we are creating:

In [ ]:
# Loss function
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation to calculate the predictions
    z1 = X.dot(W1) + b1
    a1 = ## YOUR CODE HERE // hint: look for numpy function "tanh()"
    z2 = ## YOUR CODE HERE // hint: take a look at z1
    # Now we compute a2 based on the softmax function, implemented like this:
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    
    # Calculate the loss
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    
    # Add regulatization term to loss.
    # We encode some preference for a certain set of weights W over others to remove the ambiguity
    # given by the fact that other solutions could be possible (i.e., we discourage large weights 
    # through an element-wise quadratic penalty over all parameters)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss

We implement a helper function to calculate the output of the network. 

It does forward propagation as defined above and returns the class with the highest probability.

In [ ]:
# Function to predict the output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = ## YOUR CODE HERE
    a1 = ## YOUR CODE HERE
    z2 = ## YOUR CODE HERE
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)

#### NN model 

Here we define the function that learns the parameters of the network through the batch gradient descent mechanism described above. 

In [ ]:
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations

def build_model(nn_hdim, num_passes=20000, print_loss=False):
    
    # 1. We initialize the weights to learn to random values
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))

    # We initialize our result
    model = {}
    
    # Gradient descent.
    for i in range(0, num_passes):

        # Forward propagation
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta3 = probs
        delta3[range(num_examples), y] -= 1
        dW2 = ## YOUR CODE HERE // hint: take a look at the formulas above. Also, you can compute the transpose of a matrix like this: x.T, where "x" is your matrix
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = ## YOUR CODE HERE // hint: same as with dW2
        db1 = np.sum(delta2, axis=0)

        # Add regularization terms (b1 and b2 don't have regularization terms)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1

        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        # Print the loss every 1000 iterations.
        if print_loss and i % 1000 == 0:
            print("Loss after iteration %i: %f" %(i, calculate_loss(model)))
    
    return model

### A network with a hidden layer of size 3

We train our a network with a hidden layer made of 3 units.

In [ ]:
# Build a model with a 3-dimensional hidden layer
model = build_model(3, print_loss=True)

How is the result?

In [ ]:
# Plot the decision boundary
plot_decision_boundary(lambda x: predict(model, x))
plt.title("Decision Boundary for hidden layer size 3");
plt.xticks([])
plt.yticks([])
plt.show()

# Varying the hidden layer size
What happens if we vary the number of hidden units?

In [ ]:
plt.figure(figsize=(16, 32))
hidden_layer_dimensions = [1, 2, 3, 4, 5, 20, 50]

for i, nn_hdim in enumerate(hidden_layer_dimensions):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer size %d' % nn_hdim)
    model = build_model(nn_hdim)
    plot_decision_boundary(lambda x: predict(model, x))
    plt.xticks([])
    plt.yticks([])
plt.show()

A hidden layer of low dimensionality captures very well the general trend of our data, but higher dimensionalities tend to overfit the data.

#### Generalization problem!!!

The network memorizes the training set but is not able to correctly classify a different set of data from a similar dataset

### NEXT: Use the 'tensorflow playground' interactive tool to get a deeper understanding of how these neural networks perform under different conditions: https://playground.tensorflow.org

## Part II: Applying MLP with `scikit-learn`

**Exercise**: Build a MLP that correctly classifies handwritten digits receiving as input the raw pixels from scanned images of digits. As an input to our network we will use the intensities of the image pixels. The dataset we are going to use contains a series of 8x8 greyscale images.

In [ ]:
# These are all the modules you will need to build and train your network
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Load the digits dataset
digits = load_digits()

First take a look at the content of the dataset

In [ ]:
print(digits.keys())

* The `images` attribute contains the 8x8 images 
* The `data` attribute contains the features that can be used to classify the digits (flattened version of the `images` attribute)
* The `target_names` contains the names of the classes
* The `DESCR` attribute stores a description of the dataset
* The `target` attribute provides the ground truth of the dataset (the correct class)

You can read the description accessing the `DESCR` variable:

In [ ]:
print(digits['DESCR'])

* How many instances are contained in the dataset?
* How many attributes?
* How many classes?

but it's better to verify the real content of the dataset by yourself

In [ ]:
# Nr of instances (they are far less than what is reported in the DESCRiption)
n_instances = len(digits.images)
print(n_instances)

n_instances = np.shape(digits.data)[0]
print(n_instances)

# Nr of features
n_attributes = np.shape(digits.data)[1]
print(n_attributes)

# Nr of classes
n_classes = len(digits.target_names)
print(n_classes)

In [ ]:
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[4:8]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

#### Input units: flattening the dataset
The input layer of your MLP contains units encoding values of the input pixels.
To apply the MLP classifier to your data, you need to flatten the images, that is you convert each 8x8 matrix to a 64 units vector. (NB. The dataset already contains a flattened version of the data (`data`) but here we will use the `images` instead).

You can use the `reshape` function to make the conversion (check the online help if you don't know how the `reshape` function works).

In [ ]:
data = digits.images.reshape(( n_instances, -1 ))

In [ ]:
data.shape

#### Divide the data into a training and a test set

Split the data into training and test sets. This can be done using the `train_test_split` function from the `model_selection` module.

[Hint: you can set the size of your test dataset through `test_size` in the range 0-1]

In [ ]:
from sklearn.model_selection import train_test_split

train_test_split?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, digits.target)

Check the size of your training and test sets

In [ ]:
print('Training', X_train.shape)
print('Test', X_test.shape)

#### Scale your dataset

MLP is sensitive to feature scaling, so it is highly recommended that you scale your data (i.e., having mean 0 and std 1).

Apply some scaling (normalization) to your data to improve the convergence of the network to a solution.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a new scaler
scaler = StandardScaler()

# Prepare the scaler based on the training data
scaler.fit(X_train)

In [ ]:
X_train[0,:]

In [ ]:
# Now apply the transformations to your data

X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
X_train[0,:]

### Train your model
With `scikit-learn` this becomes a very easy process. At first we import the MultiLayer Perceptron Classifier model from the `neural-network` library.

In [ ]:
from sklearn.neural_network import MLPClassifier

Create an instance of the network. At this stage you can define different parameters of your model (to know more how the function works just type `MLPClassifier?` for its documentation)

In [ ]:
MLPClassifier?

There are quite some parameters that you can set. For this example we are just going to define the size of the hidden layers and leave the rest to the default values.
I.e.:
* `activation` : {'identity', 'logistic', 'tanh', 'relu'}, default 'relu'
    Activation function for the hidden layer.

    * 'identity', no-op activation,
      returns f(x) = x

    * 'logistic', the logistic sigmoid function,
      returns f(x) = 1 / (1 + exp(-x)).

    * 'tanh', the hyperbolic tan function,
      returns f(x) = tanh(x).

    * 'relu', the rectified linear unit function,
      returns f(x) = max(0, x)
      
* `solver`: the solver for weight optimization (i.e., gradient descent)

* `learning_rate` : {'constant', 'invscaling', 'adaptive'}, default 'constant'
    Learning rate schedule for weight updates.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000)

#### Learn your training set (fit your data)

In [ ]:
mlp.fit(X_train, y_train)

#### Test your model on the remaining data

In [ ]:
predictions = mlp.predict(X_test)

In [ ]:
print('Pred\t True')
for i in range(20):
    print('%d\t %d' % (predictions[i], y_test[i]))

#### And check the results

Create a classification report and a confusion matrix

In [ ]:
c_report = classification_report(y_test, predictions)

print("Classification report for classifier %s:\n%s\n"
      % (mlp, c_report))

cfn_matrix = confusion_matrix(y_test, predictions)

print("Confusion matrix:\n%s" % cfn_matrix)

#### Plot the confusion matrix


In [ ]:
import itertools 
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:1.2f}".format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure()
plot_confusion_matrix(cfn_matrix, classes=digits.target_names, normalize=True)

Let's plot the first four examples of the test set and the their predicted label

In [ ]:
images_and_predictions = list(zip(np.reshape(X_test,(450,8,-1)), predictions))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Predict.: %i' % prediction)

#### Visualize the weights of the MLP 
In general it is not easy to make sense of the coefficients generated by a MLP, but for some tasks looking at the learned coefficients can provide insights into the learning behavior of the network. The plot below shows for each hidden unit what is the configuration of weights that makes the unit activate.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,6))
im1 = ax.matshow(np.transpose(mlp.coefs_[1]), cmap=plt.get_cmap("gray"))
cax = fig.add_axes([.92, .2, .01, .6])
fig.colorbar(im1, cax=cax, orientation='vertical')
ax.set_xlabel('Input Layer', fontsize=18)
ax.set_ylabel('Output Layer', fontsize=18)
ax.set_title('Weights matrix', fontsize=18)

 We can reshape the input weights back into the original 8x8 form of the input images and plot the resulting image.

In [ ]:
fig, axes = plt.subplots(5,2, figsize=(10,6))

vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.imshow(coef.reshape(8, 8), cmap=plt.cm.gray, vmin=.5 * vmin,
               vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

plt.suptitle('Hidden units', fontsize=18)

For instance, for the sixth hidden  neuron (plot below) what are the digits you would expect will make it fire?

In [ ]:
hidden = np.transpose(mlp.coefs_[0])[3]  # Pull weightings on inputs to the 2nd neuron in the first hidden layer

fig, ax = plt.subplots(1, figsize=(5,5))
ax.imshow(np.reshape(hidden, (8,8)), cmap=plt.get_cmap("gray"), vmin=.5 * vmin, vmax=.5 * vmax, aspect="auto")
ax.set_title('Hidden unit weights', fontsize=14)

### Hyper-parameters optimization

A learning curve shows the validation and training score of an estimator for varying numbers of training samples. It is a tool to find out how much we benefit from adding more training data.

A cross-validation generator splits the whole dataset *k* times in training
and test data. Subsets of the training set with varying sizes will be used
to train the estimator and a score for each training subset size and the
test set will be computed. Afterwards, the scores will be averaged over
all *k* runs for each training subset size.

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

# Split and shuffle your training set in different training and test sets for cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

# create an instance of your classifier
mlp2 = MLPClassifier(hidden_layer_sizes=(10,))

# Set the different sizes for the training data
train_sizes = np.linspace(.1, 1.0, 5)

# Run the crossvalidation
train_sizes, train_scores, test_scores = learning_curve(mlp2, X_train, y_train, 
                                            cv=cv, train_sizes=train_sizes, n_jobs=-1)

# Calculate statistics
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Plot the results
plt.figure()
plt.grid()
plt.title("Learning curves")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

plt.legend(loc="best")

It is sometimes useful plot the influence of a single hyperparameter on the training score and the validation score to find out whether the estimator is overfitting or underfitting for some hyperparameter values.
The `model-selection` package offers an additional function to test your model which is called `validation-curve`.
If the training score and the validation score are both low, the estimator will be underfitting. If the training score is high and the validation score is low, the estimator is overfitting and otherwise it is working very well.

## **Exercise:** Now try to use the `MLPClassifier` to identify breast cancer based on a real dataset!

Import the 'cancer' dataset from the avialable datasets

In [ ]:
# Import your dataset
from sklearn.datasets import load_breast_cancer

# Load the dataset
data = load_breast_cancer()
print(data)

Check the content of the object you just created

In [ ]:
data.keys()

Check the description of the dataset

In [ ]:
print(data['DESCR'])

Split the data into training and test sets by using the `train_test_split` function, just as before.

**Hint**: data['data'] contains the input data, whereas data['target'] contains the classification labels.

In [ ]:
X_train, X_test, y_train, y_test = ## YOUR CODE HERE

Check the size of your training and test sets

In [ ]:
print('Training', X_train.shape)
print('Test', X_test.shape)

Scale your data using `StandardScaler()`, just as before

In [ ]:
scaler = StandardScaler()

# Prepare the Scaler based on the training data
## YOUR CODE HERE

# Now apply the transformations to your data

X_train = ## YOUR CODE HERE
X_test  = ## YOUR CODE HERE

Plot the dataset in a 3D space using the first 3 features

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_train[:,10], X_train[:,11], X_train[:,12], c=y_train, marker='o')

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')

ax.view_init(30, 120)
plt.show()

### Build and train your `MLPClassifier` model
For instance, set it at 20 units per layer, and two hidden layers

In [ ]:
mlp = ## YOUR CODE HERE // Also, set the maximum number of iterations at 1000

# Call the proper function to train your MLP
## YOUR CODE HERE

### Test your model on the remaining data

In [ ]:
predictions = ## YOUR CODE HERE

In [ ]:
print(predictions)

### Evaluate the results

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(14,6))
ax = fig.add_subplot(121, projection='3d')

ax.scatter(X_test[:,0], X_test[:,1], X_test[:,2], c=y_test, marker='o', s=40)

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.set_title('Test data')
ax.view_init(30, 120)

ax1 = fig.add_subplot(122, projection='3d')
ax1.scatter(X_test[:,0], X_test[:,1], X_test[:,2], c=predictions+y_test, marker='o', s=40)

ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')
ax1.set_zlabel('Feature 3')
ax1.set_title('Predicted data (Misclassified examples in green)')

ax1.view_init(30, 120)
plt.show()

### Plot the confusion matrix

In [ ]:
cnf_matrix = confusion_matrix(y_test, predictions)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['WDBC-Malignant', 'WDBC-Benign'], normalize=True)

In [ ]:
print(classification_report(y_test, predictions, target_names=['WDBC-Malignant', 'WDBC-Benign']))